In [1]:
import sys
import os 

sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch")
sys.path.insert(0, "/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/eval")

print(sys.path)

['/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/eval', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch', '/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/analysis/clusters_units', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python38.zip', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/lib-dynload', '', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages', '/gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch', '/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages/IPython/extensions', '/gpfs7kw/linkhome/rech/genini01/ucv88ce/.ipython']


In [2]:
import glob
import pickle
from tqdm import tqdm

In [3]:
import pyximport
pyximport.install()

import json
from os.path import exists, splitext, basename
from pathlib import Path

import torch
from cpc.criterion.clustering.clustering import loadClusterModule
import cpc.eval.eval_ABX

from cpc.eval.eval_ABX_clustering import ClusteringFeatures


from cpc.feature_loader import buildFeature, FeatureModule, loadModel
import argparse


/gpfswork/rech/ank/ucv88ce/miniconda3/envs/inftrain/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


In [4]:
#find a way to use jupyter notebook with gpu
#srun --ntasks=1 --cpus-per-task=10 --gres=gpu:1 --hint=nomultithread --time=01:00:00 --pty bash 

In [ ]:
#FOR FRENCH

%run /gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/clustering/clustering_quantization.py --file_extension "wav"  \
"../../experiments/checkpoints/inftrain/FR/3200h/00/kmeans100/checkpoint_last.pt" \
"/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr" \
"../../experiments/eval/inftrain/FR/3200h/00/kmeans100/"

In [ ]:
#FOR ENGLISH

%run /gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/clustering/clustering_quantization.py --file_extension "wav"  \
"../../experiments/checkpoints/inftrain/EN/3200h/00/kmeans100/checkpoint_last.pt" \
"/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en" \
"../../experiments/eval/inftrain/EN/3200h/00/kmeans100/"

In [21]:
#ENGLISH ON LS

#FOR ENGLISH 

%run /gpfsdswork/projects/rech/ank/ucv88ce/repos/CPC_torch/cpc/clustering/clustering_quantization.py --file_extension "wav"  \
"../../experiments/checkpoints/inftrain/EN/3200h/00/kmeans50/checkpoint_last.pt" \
"/gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean" \
"../../experiments/eval/inftrain/EN/3200h/00/kmeans50/LS-CLEAN"


Quantizing data from /gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean

Looking for all wav files in /gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean with speakerLevel 1
Loaded from cache /gpfsssd/scratch/rech/ank/ucv88ce/data/LibriSpeech/test-clean/_seqs_cache.txt successfully
Done! Found 2620 files and 87 speakers!
Loading args from ../../experiments/checkpoints/inftrain/EN/3200h/00/kmeans50

Clutering args:
{
    "DPLambda": 11,
    "DPMean": false,
    "MAX_ITER": 300,
    "batchSizeGPU": 200,
    "centroidLimits": null,
    "debug": false,
    "dimReduction": null,
    "dirOutput": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/kmeans50",
    "encoder_layer": false,
    "extension": "wav",
    "getDistanceEstimation": false,
    "level_gru": 2,
    "load": null,
    "nClusters": 50,
    "nGroups": 1,
    "pathCheckpoint": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/3200h/00/cpc_small/checkpoint_25.pt",
    "pathDB": "/gpfsssd/scratc

  0% (7 of 2620) |                       | Elapsed Time: 0:00:00 ETA:   0:00:42

CPC FeatureMaker loaded!

Quantizing audio files...


100% (2620 of 2620) |####################| Elapsed Time: 0:01:07 Time:  0:01:07


...done 2620 files in 67.65155124664307 seconds.

Saving outputs to ../../experiments/eval/inftrain/EN/3200h/00/kmeans50/LS-CLEAN/quantized_outputs.txt


## LINES BELOW WERE BEFORE USING THE QUANTIZATION SCRIPT. NOT USEFUL ANYMORE.

In [5]:
wav_fr = [x for x in glob.glob("/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/fr/*.wav")]
wav_en = [x for x in glob.glob("/gpfsscratch/rech/cfs/commun/cv21_ABX/raw_dataset/en/*.wav")]

In [6]:
def get_units(kmeans_cp, wav_list, output_pkl=None):
    audio2units={}
    #kmeans_cp="/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/50h/00/kmeans50/checkpoint_last.pt"
    FeatureMaker = ClusteringFeatures(
                kmeans_cp,
                soft_clustering=False,
                encoder_layer=False,
                keepHidden=True,
                group_modes="onehot",
                onehot_dict=None,
    )
    for audio in tqdm(wav_list):
        audio_name = os.path.basename(audio)
        unit_list=[]
        T=FeatureMaker.feature_function(audio)
        a_tuple = (T == 1).nonzero(as_tuple=True)
        a=a_tuple[2]
        for i in range(len(a)):
            unit_list.append(a[i].item())
        audio2units[audio_name]=unit_list
    
    if output_pkl:
        with open(output_pkl, 'wb') as handle:
            pickle.dump(audio2units, handle)
    return audio2units

        

In [7]:
#HERE IS CORRECT



#exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/*/*/*/kmeans50")]
#exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/EN/*/*/kmeans50")]
exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/*/3200h/*/kmeans100")]


prefix = "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/"
for exp  in exp_list:
    exp_path = exp[len(prefix):]
    print("EXP_PATH = ", exp_path)
    for lang in ["en", "fr"]:
        if lang == "fr":
            wav_list = wav_fr
        elif lang == "en":
            wav_list = wav_en
        kmeans_cp = os.path.join("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/", exp_path, "checkpoint_last.pt")
        output_path=os.path.join("/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/experiments/eval/inftrain", exp_path)
        output_pkl=os.path.join(output_path, lang+"_units.pkl")

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        if not os.path.exists(output_pkl):
            get_units(kmeans_cp, wav_list, output_pkl=output_pkl)




EXP_PATH =  EN/3200h/00/kmeans100
EXP_PATH =  FR/3200h/00/kmeans100
Loading args from /gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/3200h/00/kmeans100/args.json

Clutering args:
{
    "DPLambda": 11,
    "DPMean": false,
    "MAX_ITER": 300,
    "batchSizeGPU": 200,
    "centroidLimits": null,
    "debug": false,
    "dimReduction": null,
    "dirOutput": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/3200h/00/kmeans100",
    "encoder_layer": false,
    "extension": "wav",
    "getDistanceEstimation": false,
    "level_gru": 2,
    "load": null,
    "nClusters": 100,
    "nGroups": 1,
    "pathCheckpoint": "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/3200h/00/cpc_small/checkpoint_20.pt",
    "pathDB": "/gpfsssd/scratch/rech/cfs/commun/families/FR/3200h/00",
    "perIterSize": 1406,
    "recursionLevel": 2,
    "save": true,
    "save_last": 5,
    "seqList": null,
    "sizeWindow": 10240,
    "train_mode": false
}
--------------------------------------------------
L

FileNotFoundError: [Errno 2] No such file or directory: '/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/3200h/00/kmeans100/checkpoint_last.pt'

In [ ]:
#HERE IS CORRECT ( FR)



#exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/*/*/*/kmeans50")]
exp_list = [x for x in glob.glob("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/FR/*/*/kmeans50")]

prefix = "/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/"
for exp  in exp_list:
    exp_path = exp[len(prefix):]
    print("EXP_PATH = ", exp_path)
    for lang in ["en", "fr"]:
        if lang == "fr":
            wav_list = wav_fr
        elif lang == "en":
            wav_list = wav_en
        kmeans_cp = os.path.join("/gpfsssd/scratch/rech/cfs/commun/InfTrain_models/", exp_path, "checkpoint_last.pt")
        output_path=os.path.join("/gpfsdswork/projects/rech/ank/ucv88ce/projects/MultilingualCPC/experiments/eval/inftrain", exp_path)
        output_pkl=os.path.join(output_path, lang+"_units.pkl")

        if not os.path.exists(output_path):
            os.makedirs(output_path)

        if not os.path.exists(output_pkl):
            get_units(kmeans_cp, wav_list, output_pkl=output_pkl)




In [63]:
#need to double check. Function that is not thoroughly checked but seems to be working
#this should lalow us to get average length

#should obviously be done on a lit of file. 


units={x:[] for x in range(50)}
T=FeatureMaker.feature_function(audio_file)
a_tuple = (T == 1).nonzero(as_tuple=True)
a=a_tuple[2]
curr_unit=0
for i in range(len(a)):
    curr_unit+=1
    if i+1 == len(a):#edge case
        units[a[i].item()].append(curr_unit)
        curr_unit=0
    elif a[i+1].item() != a[i].item():
        units[a[i].item()].append(curr_unit)
        curr_unit=0

        